In [5]:
import urllib.request,sys,time
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re
import os.path
import glob
import os
from unidecode import unidecode

ModuleNotFoundError: No module named 'unidecode'

In [12]:
def extract_news(start_date = '1402/07/01',end_date = '1402/08/01',service_id=0,section_id=0,subject_id = 0):
    page_number = 1
    url = 'https://www.asriran.com/fa/archive?service_id='+str(service_id)+'&sec_id='+str(section_id)+'&cat_id='+str(subject_id)+'&rpp=100&from_date='+start_date+'&to_date='+end_date+'&p='
    while True:

        if (os.path.isfile('.\\data'+str(page_number)+'.csv')):
            print('there is already page '+str(page_number))
            page_number += 1
            continue


        print(page_number)
        upperframe=[] 
        url1 = url + str(page_number)
        page = requests.get(url1)
        # print(page.text)
        soup = BeautifulSoup(page.text, "html.parser")
        # print(page.text)
        # with open('output'+str(page_number)+'.html', "w",encoding="utf-8") as file:
        #     file.write(str(soup))


        articles=soup.find_all('article',attrs={'vizhe_cv col-xs-12 col-sm-6'})
        if len(articles)==0 :
            print('finished')
            break
        print('page '+ str(page_number))
       
        for article in articles:
            title = article.find("a",attrs={'class':"vizhe_title"}).text.strip()
            link = article.find("a").get('href').strip()
            link = 'https://www.asriran.com' + link

            summary = article.find("div",attrs={'class':"vizhe_lead"}).text.strip()
            date = article.find("span",attrs={'class':"tarikh_archive"}).text.strip()

            url2 = link
            page2 = requests.get(url2)
            soup2 = BeautifulSoup(page2.text, "html.parser")


            # IF THE PAGE IS GONE
            ERROR = soup2.find("div",attrs={'class':"error_container"})
            if ERROR :
                subject = ''
                body = 'صفحه درخواستی شما موجود نمی باشد'
                news_id = ''
                short_link = ''
                img_path=''
                tag_word=''
                frame = [news_id,date,subject,title,summary,body,short_link,img_path,tag_word]
                upperframe.append(frame)
                continue # jump to the next article


            subject = soup2.find("div",attrs={'class':"news_path"}).find_all("a")[-1].text
            # if  (subject != None):
                # subject = subject.find_all("a")[-1].text

            # body = soup2.find("div",attrs={'class':"body"}).text.strip()[:-155]
            body = soup2.find("div",attrs={'class':"body"}).text.strip()[:-155]
            # if  (body != None):
            #     body = body.text.strip()[:-155]

            img_path = soup2.find("img",attrs={'class':"lead_image"})
            if  (img_path != None):
                img_path = img_path.get('src').strip()

            news_id = soup2.find("div",attrs={'class':"news_nav news_id_c"}).text.strip()
            news_id = int(re.findall(r'\d+', news_id)[0])
            news_id = (str(news_id))
            # if  (news_id != None):
                # news_id = news_id.text.strip()

                # news_id = int(re.findall(r'\d+', news_id)[0])
                # news_id = (str(news_id))
                # news_id = unidecode(u"۰۱۲۳۴۵۶۷۸۹")

            tags_list = []
            tags_items = soup2.find_all("a", class_="tags_item")
            if tags_items:
                for tag_item in tags_items:
                    tag_text = tag_item.get_text(strip=True)
                    tags_list.append(tag_text)

            short_link = ""
            if soup2.find("div",attrs={'class':"short-link row"}).find("a"):
                short_link = soup2.find("div",attrs={'class':"short-link row"}).find("a").get('href').strip()
                short_link = 'https://www.asriran.com' + short_link

            # df = pd.DataFrame(columns=['tags'])
            #
            # # Append the list of tags to the DataFrame
            # df['tags'] = [tags_list]
            
            frame = [news_id,date,subject,title,summary,body,short_link,img_path,tags_list]
            upperframe.append(frame)
            # upperframe[-1].append(tags_list)

        data=pd.DataFrame(upperframe, columns=['News_ID','Date','Subject','Title','Summary','Body','Short_link','img_path','tag_list'])
        data.to_csv('data'+str(page_number)+'.csv', encoding='utf-8', index=False)
        page_number += 1

    #data=pd.DataFrame(upperframe, columns=['News_ID','Date','Subject','Title','Summary','Body','Short_link'])

    return print("DONE")

extract_news(start_date = '1402/07/28',end_date = '1402/08/01',service_id=0,section_id=0,subject_id = 0)

1
page 1
2
page 2
3
page 3
4
page 4
5
page 5
6
page 6
7
finished
DONE


In [13]:
# setting the path for joining multiple files
#files = os.path.join(".\\", "data*.csv")
# https://sparkbyexamples.com/pandas/pandas-read-multiple-csv-files/

# list of merged files returned
files = glob.glob('.\\data*.csv')

# joining files with concat and read_csv
df = pd.concat(map(pd.read_csv, files), ignore_index=True)

# convert df to csv
df.to_csv('data.csv', encoding='utf-8', index=False)

In [14]:
df.head(5)

,News_ID,Date,Subject,Title,Summary,Body,Short_link,img_path,tag_list
0,912803,۱۴۰۲/۰۸/۰۱,اجتماعی,بیشترین جستجوی گوگل ایرانیان در مهر,مهر امسال، عجیب بود. مرگ سه هنرمند باعث شد که ...,کانال عصر ایران در تلگرام ...,https://www.asriran.com/003pSd,https://cdn.asriran.com/files/fa/news/1402/8/1...,"['گوگل', 'ایرانیان']"
1,912802,۱۴۰۲/۰۸/۰۱,بین الملل,بایدن : گفت‌وگو برای آتش‌ بس میان حماس و اسرائ...,بایدن در کنفرانسی خبری در پاسخ به سوالی در راب...,جو بایدن، رئیس‌جمهوری آمریکا گفت‌وگو برای حصول...,https://www.asriran.com/003pSc,https://cdn.asriran.com/files/fa/news/1402/8/1...,"['بایدن', 'غزه']"
2,912801,۱۴۰۲/۰۸/۰۱,اجتماعی,کنکور اختصاصی تربیت معلم اردیبهشت ۱۴۰۳ برگزار ...,در اردیبهشت ۱۴۰۳ برای اولین بار کنکور تربیت مع...,وزیر آموزش و پرورش در جلسه هیأت امنای دانشگاه ...,https://www.asriran.com/003pSb,https://cdn.asriran.com/files/fa/news/1402/8/1...,"['کنکور', 'معلم']"
3,912800,۱۴۰۲/۰۸/۰۱,بین الملل,حماس دو اسیر دیگر را آزاد کرد,رویترز نوشت که این خبر توسط گردان القسام - از ...,حماس امروز (دوشنبه) از آزادی دو اسیر دیگر خبر ...,https://www.asriran.com/003pSa,https://cdn.asriran.com/files/fa/news/1402/8/1...,"['حماس', 'آمریکا', 'اسیر']"
4,912799,۱۴۰۲/۰۸/۰۱,سیاسی,امیرعبداللهیان: آمریکا به ایران پیام داده و گف...,وزیر امور خارجه: آقای بایدن ریاکاری را متوقف ک...,حسین امیرعبداللهیان درباره پیام آمریکا به ایرا...,https://www.asriran.com/003pSZ,https://cdn.asriran.com/files/fa/news/1402/8/1...,"['آمریکا', 'ایران']"


In [12]:
# number of articles
len(df. index)

4376

In [ ]:
import csv

def get_unique_values(csv_file_path, column_name):
    unique_values = set()
    with open(csv_file_path, 'r', encoding='utf-8') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            unique_values.add(row[column_name])
    return list(unique_values)

# Example usage
csv_file_path = 'data.csv'
column_name = 'Subject'
unique_subjects = get_unique_values(csv_file_path, column_name)
print(unique_subjects)
